# 数据清洗
发现mesh有的是空的，清洗一下

In [1]:
import json 
with open('/home/edre/dtw_files/DeepCAD/data/cad_data/train_val_test_split.json','r') as f :
    data=json.load(f)

In [2]:
import os
import trimesh
import json

train_num = 800
test_num = 100

# 创建目标目录的基本结构
os.makedirs('data/full_data/cad_vec', exist_ok=True)
os.makedirs('data/full_data/cad_pcd', exist_ok=True)

def load_and_validate_mesh(path):
    try:
        mesh = trimesh.load(path, force='mesh')
        if mesh.is_empty:
            print(f"Warning: The mesh at {path} is empty.")
            return False
        return True
    except Exception as e:
        print(f"Failed to load mesh at {path}: {str(e)}")
        return False

def create_symlinks_and_collect_paths(file_paths, target_folder, num_samples):
    paths_collected = []
    count = 0
    for path in file_paths:
        h5_data = os.path.join('data/cad_data/cad_vec', path + '.h5')
        mesh_data = os.path.join('data/cad_data/pc_cad', path + '.ply')
        
        if load_and_validate_mesh(mesh_data) and os.path.exists(h5_data) and os.path.exists(mesh_data):
            # 创建目标路径，保留原始的目录结构
            target_h5_path = os.path.join('data/full_data/cad_vec', path+'.h5')
            target_mesh_path = os.path.join('data/full_data/pc_cad',path+'.ply' )

            # 创建必要的目录
            os.makedirs(os.path.dirname(target_h5_path), exist_ok=True)
            os.makedirs(os.path.dirname(target_mesh_path), exist_ok=True)

            # 创建符号链接，先检查是否已存在
            if os.path.exists(target_h5_path):
                os.remove(target_h5_path)
            if os.path.exists(target_mesh_path):
                os.remove(target_mesh_path)

            os.symlink(os.path.abspath(h5_data), target_h5_path)
            os.symlink(os.path.abspath(mesh_data), target_mesh_path)
            
            paths_collected.append(path)
            
            count += 1
            # if count == num_samples:
            #     break
    return paths_collected

# 收集路径
train_paths = create_symlinks_and_collect_paths(data['train'], 'train', train_num)
test_paths = create_symlinks_and_collect_paths(data['test'], 'test', test_num)
val_paths = create_symlinks_and_collect_paths(data['validation'], 'test', test_num)

# 写入JSON文件
split_data = {
    'train': train_paths,
    'test': test_paths,
    'validation':val_paths
}

with open('data/full_data/train_test_split.json', 'w') as json_file:
    json.dump(split_data, json_file, indent=4)


Failed to load mesh at data/cad_data/pc_cad/0086/00866760.ply: string is not a file: data/cad_data/pc_cad/0086/00866760.ply
Failed to load mesh at data/cad_data/pc_cad/0022/00222961.ply: string is not a file: data/cad_data/pc_cad/0022/00222961.ply
Failed to load mesh at data/cad_data/pc_cad/0083/00837927.ply: string is not a file: data/cad_data/pc_cad/0083/00837927.ply
Failed to load mesh at data/cad_data/pc_cad/0070/00704820.ply: string is not a file: data/cad_data/pc_cad/0070/00704820.ply
Failed to load mesh at data/cad_data/pc_cad/0087/00870042.ply: string is not a file: data/cad_data/pc_cad/0087/00870042.ply
Failed to load mesh at data/cad_data/pc_cad/0070/00703301.ply: string is not a file: data/cad_data/pc_cad/0070/00703301.ply
Failed to load mesh at data/cad_data/pc_cad/0081/00817507.ply: string is not a file: data/cad_data/pc_cad/0081/00817507.ply
Failed to load mesh at data/cad_data/pc_cad/0033/00333559.ply: string is not a file: data/cad_data/pc_cad/0033/00333559.ply
Failed t

# 提前处理好点云